## **Exercise - Perform classification with alternative metrics**

### **Classification Metrics**

In the last notebook we fit binary classifier to predict whether patients were diabetic or not. We used accuracy as a measure of how well the model performed, but accuracy isn't everything. In this notebook, we will look at alternatives to accuracy that can be much more useful in machine learning.

### **Alternative metrics for binary classifiers**

Accuracy seems like a sensible metric to evaluate (and to a certain extent it is), but you need to be careful about drawing too many conclusions from the accuracy of a classifier. Remember that it's simply a measure of how many cases were predicted correctly. Suppose only 3% of the population is diabetic. You could create a classifier that always just predicts 0, and it would be 97% accurate - but not terribly helpful in identifying patients with diabetes!

Fortunately, there are some other metrics that reveal a little more about how our classification model is performing.


In [ ]:
# Load the required packages and make them available in your current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(janitor)
  library(tidymodels)
})


# Read the csv file into a tibble
diabetes <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/diabetes.csv", show_col_types = FALSE)


# Clean data a bit
diabetes_select <- diabetes %>%
  # Encode Diabetic as category
  mutate(Diabetic = factor(Diabetic, levels = c("1","0"))) %>% 
  # Drop PatientID column
  select(-PatientID) %>% 
  # Clean column names
  clean_names()


# Split data into 70% for training and 30% for testing
set.seed(2056)
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.70)


# Extract the data in each split
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)


# Make a model specifcation
logreg_spec <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


# Train a logistic regression model
logreg_fit <- logreg_spec %>% 
  fit(diabetic ~ ., data = diabetes_train)


# Make predictions then bind them to the test set
results <- diabetes_test %>% select(diabetic) %>% 
  bind_cols(logreg_fit %>% predict(new_data = diabetes_test))


# Calculate accuracy: proportion of data predicted correctly
accuracy(data = results, truth = diabetic, estimate = .pred_class)


One performance metric associated with classification problems is the [`confusion matrix`](https://wikipedia.org/wiki/Confusion_matrix). A confusion matrix describes how well a classification model performs by tabulating how many examples in each class were correctly classified by a model. In our case, it will show you how many cases were classified as negative (0) and how many as positive (1); the confusion matrix also shows you how many were classified into the *wrong* categories.

The [`conf_mat()`](https://tidymodels.github.io/yardstick/reference/conf_mat.html) function from yardstick calculates this cross-tabulation of observed and predicted classes.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = diabetic, estimate = .pred_class)


Awesome!

Let's interpret the confusion matrix. Our model is asked to classify cases between two binary categories, category `1` for patients who tested positive for diabetes and category `0` for patients who tested negative.

-   If your model predicts a patient as `1` (positive) and they belong to category `1` (positive) in reality we call this a `true positive`, shown by the top left number `897`.

-   If your model predicts a patient as `0` (negative) and they belong to category `1` (positive) in reality we call this a `false negative`, shown by the bottom left number `657`.

-   If your model predicts a patient as `1` (positive) and they belong to category `0` (negative) in reality we call this a `false positive`, shown by the top right number `293`.

-   If your model predicts a patient as `0` (negative) and they belong to category `0` (negative) in reality we call this a `true negative`, shown by the bottom right number `2653`.

Our confusion matrix can thus be expressed in the following form:

| Truth |
|:-----:|

|               |                  |                   |
|:-------------:|:----------------:|:-----------------:|
| **Predicted** |        1         |         0         |
|       1       | $897_{\ \ \ TP}$ | $293_{\ \ \ FP}$  |
|       0       | $657_{\ \ \ FN}$ | $2653_{\ \ \ TN}$ |

Note that the correct (*`true`*) predictions form a diagonal line from top left to bottom right - these figures should be significantly higher than the *false* predictions if the model is any good.

As you might have guessed it's preferable to have a larger number of true positives and true negatives and a lower number of false positives and false negatives, which implies that the model performs better.

The confusion matrix is helpful since it gives rise to other metrics that can help us better evaluate the performance of a classification model. Let's go through some of them:

🎓 Precision: `TP/(TP + FP)` defined as the proportion of predicted positives that are actually positive. Also called [positive predictive value](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value")

🎓 Recall: `TP/(TP + FN)` defined as the proportion of positive results out of the number of samples which were actually positive. Also known as `sensitivity`.

🎓 Specificity: `TN/(TN + FP)` defined as the proportion of negative results out of the number of samples which were actually negative.

🎓 Accuracy: `TP + TN/(TP + TN + FP + FN)` The percentage of labels predicted accurately for a sample.

🎓 F Measure: A weighted average of the precision and recall, with best being 1 and worst being 0.

Tidymodels provides yet another succinct way of evaluating all these metrics. Using `yardstick::metric_set()`, you can combine multiple metrics together into a new function that calculates all of them at once.


In [ ]:
# Combine metrics and evaluate them all at once
eval_metrics <- metric_set(ppv, recall, accuracy, f_meas)
eval_metrics(data = results, truth = diabetic, estimate = .pred_class)


Using the precision (ppv) metric, we are able to answer the question:

-   Of all the patients the model predicted are diabetic, how many are actually diabetic?

Using the recall metric, we are able to answer the question:

-   Of all the patients that are actually diabetic, how many did the model identify?

Great job, we just made predictions and evaluated them using a number of metrics.

Until now, we've considered the predictions from the model as being either 1 or 0 class labels. Actually, things are a little more complex than that. Statistical machine learning algorithms, like logistic regression, are based on `probability`; so what actually gets predicted by a binary classifier is the probability that the label is true ($P(y)$) and the probability that the label is false ($1-P(y)$). A threshold value of 0.5 is used to decide whether the predicted label is a `1` ($P(y)>0.5$) or a `0` ($P(y)<=0.5$). Let's see the probability pairs for each case:


In [ ]:
# Predict class probabilities and bind them to results
results <- results %>% 
  bind_cols(logreg_fit %>% 
              predict(new_data = diabetes_test, type = "prob"))

# Print out the results
results %>% 
  slice_head(n = 10)


The decision to score a prediction as a 1 or a 0 depends on the threshold to which the predicted probabilities are compared. If we were to change the threshold, it would affect the predictions; and therefore change the metrics in the confusion matrix. A common way to evaluate a classifier is to examine the *true positive rate* (which is another name for recall) and the *false positive rate* (1 - specificity) for a range of possible thresholds. These rates are then plotted against all possible thresholds to form a chart known as a *received operator characteristic (ROC) chart*, like this:



In [ ]:
# Make a roc_chart
results %>% 
  roc_curve(truth = diabetic, .pred_1) %>% 
  autoplot()


The ROC chart shows the curve of the true and false positive rates for different threshold values between 0 and 1. A perfect classifier would have a curve that goes straight up the left side and straight across the top. The diagonal line across the chart represents the probability of predicting correctly with a 50/50 random prediction; so you obviously want the curve to be higher than that (or your model is no better than simply guessing!).

The area under the curve (AUC) is a value between 0 and 1 that quantifies the overall performance of the model. One way of interpreting AUC is as the probability that the model ranks a random positive example more highly than a random negative example. The closer to 1 this value is, the better the model. Once again, Tidymodels includes a function to calculate this metric: `yardstick::roc_auc()`


In [ ]:
# Compute the AUC
results %>% 
  roc_auc(diabetic, .pred_1)


### Data preprocessing with recipes

In this case, the ROC curve and its AUC indicate that the model performs better than a random guess which is not bad considering we performed very little preprocessing of the data.

In practice, it's common to perform some preprocessing of the data to make it easier for the algorithm to fit a model to it. There's a huge range of preprocessing transformations you can perform to get your data ready for modeling, but we'll limit ourselves to a few common techniques:

-   Scaling numeric features so they're on the same scale. This prevents features with large values from producing coefficients that disproportionately affect the predictions.

-   Encoding categorical variables. For example, by using a *one hot encoding* technique you can create "*dummy*" or *indicator variables* which replace the original categorical feature with numeric columns whose values are either 1 or 0.

Tidymodels provides yet another neat package: [recipes](https://recipes.tidymodels.org/)- a package for preprocessing data. Let's specify a recipe that encodes the age column then normalizes the rest of the predictor features.


In [ ]:
# Preprocess the data for modelling
diabetes_recipe <- recipe(diabetic ~ ., data = diabetes_train) %>% 
  step_mutate(age = factor(age)) %>% 
  step_normalize(all_numeric_predictors()) %>% 
  step_dummy(age)

# Print the recipe
diabetes_recipe


We just created a recipe containing an outcome and its corresponding predictors, specifying that the age variable should be converted to a categorical variable (factor), all the numeric predictors normalized and creating dummy variables for the nominal predictor (age) 🙌.

### Bundling it all together using a workflow

Now that we have a recipe and a model specification we defined previously, we need to find a way of bundling them together into an object that will first preprocess the data, fit the model on the preprocessed data and also allow for potential post-processing activities.

In Tidymodels, this convenient object is called a [`workflow`](https://workflows.tidymodels.org/) and conveniently holds your modeling components.

The [**workflows**](https://workflows.tidymodels.org/) package allows the user to bind modeling and preprocessing objects together. You can then fit the entire workflow to the data, such that the model encapsulates all of the preprocessing steps as well as the algorithm.


In [ ]:
# Redefine the model specification
logreg_spec <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")

# Bundle the recipe and model specification
lr_wf <- workflow() %>% 
  add_recipe(diabetes_recipe) %>% 
  add_model(logreg_spec)

# Print the workflow
lr_wf


After a workflow has been *specified*, a model can be `trained` using the [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function.



In [ ]:
# Fit a workflow object
lr_wf_fit <- lr_wf %>% 
  fit(data = diabetes_train)

# Print wf object
lr_wf_fit


Good job👏! We now have a trained workflow. The workflow print-out shows the coefficients learned during training.

This allows us to use the model trained by this workflow to predict labels for our test set, and compare the performance metrics with the basic model we created previously.


In [ ]:
# Make predictions on the test set
results <- diabetes_test %>% select(diabetic) %>% 
  bind_cols(lr_wf_fit %>% 
              predict(new_data = diabetes_test)) %>% 
  bind_cols(lr_wf_fit %>% 
              predict(new_data = diabetes_test, type = "prob"))

# Print the results
results %>% 
  slice_head(n = 10)


Let's take a look at the confusion matrix:



In [ ]:
# Confusion matrix for prediction results
results %>% 
  conf_mat(truth = diabetic, estimate = .pred_class)


🤩🤩 Look at those metrics!

Can we visualize these proportions? Of course, nothing is impaRsible!


In [ ]:
# Visualize conf mat
update_geom_defaults(geom = "rect", new = list(fill = "midnightblue", alpha = 0.7))

results %>% 
  conf_mat(diabetic, .pred_class) %>% 
  autoplot()


What about our other performance metrics that can be derived from the confusion matrix?



In [ ]:
# Evaluate other desired metrics
eval_metrics(data = results, truth = diabetic, estimate = .pred_class)


Finally, let's take a look at the ROC Metrics.



In [ ]:
# Evaluate ROC_AUC metrics
results %>% 
  roc_auc(diabetic, .pred_1)

# Plot ROC_CURVE
results %>% 
  roc_curve(diabetic, .pred_1) %>% 
  autoplot()


Comparing with previous predictions, the metrics look better, so clearly preprocessing the data has made a difference!

### Try a different algorithm

Now let's try a different algorithm. Previously we used a logistic regression algorithm, which is a *linear* algorithm. There are many kinds of classification algorithm we could try, including:

-   **Support Vector Machine algorithms**: Algorithms that define a *hyperplane* that separates classes.

-   **Tree-based algorithms**: Algorithms that build a decision tree to reach a prediction

-   **Ensemble algorithms**: Algorithms that combine the outputs of multiple base algorithms to improve generalizability.

This time, we'll train the model using an *ensemble* algorithm named *Random Forest* that averages the outputs of multiple random decision trees. Random forests help to reduce tree correlation by injecting more randomness into the tree-growing process. More specifically, instead of considering all predictors in the data, for calculating a given split, random forests pick a random sample of predictors to be considered for that split.

As we already have a gist of how to perform classification using Tidymodels, let's get right into specifying and fitting a random forest algorithm.


In [ ]:
# Preprocess the data for modelling
diabetes_recipe <- recipe(diabetic ~ ., data = diabetes_train) %>% 
  step_mutate(age = factor(age)) %>% 
  step_normalize(all_numeric_predictors()) %>% 
  step_dummy(age)

# Build a random forest model specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger", importance = "impurity") %>% 
  set_mode("classification")

# Bundle recipe and model spec into a workflow
rf_wf <- workflow() %>% 
  add_recipe(diabetes_recipe) %>% 
  add_model(rf_spec)

# Fit a model
rf_wf_fit <- rf_wf %>% 
  fit(data = diabetes_train)

# Make predictions on test data
results <- diabetes_test %>% select(diabetic) %>% 
  bind_cols(rf_wf_fit %>% 
              predict(new_data = diabetes_test)) %>% 
  bind_cols(rf_wf_fit %>% 
              predict(new_data = diabetes_test, type = "prob"))

# Print out predictions
results %>% 
  slice_head(n = 10)


💪 There goes our random_forest model. Is it any good? Let's evaluate its metrics!



In [ ]:
# Confusion metrics for rf_predictions
results %>% 
  conf_mat(diabetic, .pred_class)

# Confusion matrix plot
results %>% 
  conf_mat(diabetic, .pred_class) %>% 
  autoplot()


There is a considerable increase in the number of `True Positives` and `True Negatives`, which is a step in the right direction.

Let's take a look at other evaluation metrics


In [ ]:
# Evaluate other intuitive classification metrics
rf_met <- results %>% 
  eval_metrics(truth = diabetic, estimate = .pred_class)

# Evaluate ROC_AUC
auc <- results %>% 
  roc_auc(diabetic, .pred_1)

# Plot ROC_CURVE
curve <- results %>% 
  roc_curve(diabetic, .pred_1) %>% 
  autoplot

# Return metrics
list(cm_metrics = rf_met, AUC = auc, ROC_CURVE = curve)


For the sheer sake of adventure, let's make a Variable Importance Plot to see which predictor variables have the most impact in our model.



In [ ]:
# Load vip
library(vip)

# Extract the fitted model from the workflow
rf_wf_fit %>% 
  extract_fit_parsnip() %>% 
# Make VIP plot
  vip()


The VIP curve just reinforced what we put forward during data exploration. This goes to show the importance of data exploration.

As revealed by the performance metrics, the random forest model seemed to have done a great job in increasing the True Positives/Negatives and reducing the False Positives/Negatives.

### Use the model for inferencing

Now that we have a reasonably useful trained model, we can save it for use later to predict labels for new data:


In [ ]:
library(here)
# Save trained workflow in the location of this script
saveRDS(rf_wf_fit, "diabetes_rf_model.rds")


Now, we can load it whenever we need it, and use it to predict labels for new data. This is often called *`scoring`* or *`inferencing`*.

For example, lets create a simulated data set by picking a random value for each column in our test set then make predictions using the saved model.


In [ ]:
# Load the model into the current R session
loaded_model <- readRDS("diabetes_rf_model.rds")

# Create new simulated data
new_data <- map_dfc(diabetes_test, function(x){sample(x, size = 2)}) %>% 
  as_tibble()

# Make predictions
predictions <- new_data %>% 
  bind_cols(loaded_model %>% predict(new_data))

predictions
